In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df_tmp = pd.merge(df3, df1, left_on='user_id', right_on='user_id')
df = pd.merge(df_tmp, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.head()

Using TensorFlow backend.


,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,30920,9,567330,1,504423,3,32638
1,320815,15,567330,1,504423,3,32638
2,355089,11,567330,1,504423,3,32638
3,363442,9,567330,1,504423,3,32638
4,370513,14,567330,1,504423,3,32638


In [2]:
df.sort_values(['user_id', 'time'], inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
16204330,1,20,877468,1,773445,5,29455
23067470,1,20,209778,1,188507,2,9702
26781033,1,20,821396,1,724607,5,7293
22502290,1,39,1683713,1,1458878,5,14668
9504689,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
28703035,900000,57,2692144,1,2316309,5,14372
15724163,900000,71,3391205,1,2918341,5,22137
7226547,900000,72,3471208,1,2989104,5,7283
9638586,900000,74,3527098,1,3037879,18,14681


In [3]:
df.reset_index(drop=True, inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,1,20,877468,1,773445,5,29455
1,1,20,209778,1,188507,2,9702
2,1,20,821396,1,724607,5,7293
3,1,39,1683713,1,1458878,5,14668
4,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
30082766,900000,57,2692144,1,2316309,5,14372
30082767,900000,71,3391205,1,2918341,5,22137
30082768,900000,72,3471208,1,2989104,5,7283
30082769,900000,74,3527098,1,3037879,18,14681


In [4]:
df['creative_id'].max()

4445718

In [5]:
# creative_id embedding后 作为特征
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(4445719, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 11us/step


,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,-0.044800,-0.019390,-0.031851,0.030554,-0.040778
1,0.043076,-0.025291,-0.006318,0.018357,-0.021721
2,-0.043862,-0.046132,0.047778,-0.008464,0.017775
3,0.037853,0.040882,-0.035817,-0.036202,0.043400
4,-0.044382,0.048629,-0.025343,-0.041154,-0.023619
...,...,...,...,...,...
30082766,-0.046034,-0.040601,-0.019310,-0.034490,-0.038199
30082767,-0.018046,-0.011815,0.022564,-0.020459,-0.030722
30082768,-0.037631,0.018072,-0.023263,-0.007455,-0.046859
30082769,0.047910,0.031352,0.011601,-0.027340,-0.038355


In [6]:
# time click_times 直接作为特征
X = df[['user_id', 'time', 'click_times']]

In [7]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,1,20,1,-0.044800,-0.019390,-0.031851,0.030554,-0.040778
1,1,20,1,0.043076,-0.025291,-0.006318,0.018357,-0.021721
2,1,20,1,-0.043862,-0.046132,0.047778,-0.008464,0.017775
3,1,39,1,0.037853,0.040882,-0.035817,-0.036202,0.043400
4,1,40,1,-0.044382,0.048629,-0.025343,-0.041154,-0.023619


In [8]:
df['ad_id'].max()

3812200

In [9]:
# ad_id embedding后 作为特征
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(3812201, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 11us/step


,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,-0.001448,-0.009343,0.004232,-0.025161,-0.028182
1,0.033837,-0.038700,0.039421,-0.034426,-0.041416
2,0.039120,-0.046248,-0.046503,0.019543,-0.020489
3,0.005398,-0.004463,-0.022102,0.008920,0.026426
4,0.018380,-0.037460,0.043368,0.009137,0.038793
...,...,...,...,...,...
30082766,0.043033,-0.047365,-0.028866,0.014418,0.012414
30082767,0.030707,-0.000822,0.043278,-0.015158,0.002837
30082768,0.048633,-0.003637,-0.013621,0.004312,-0.010405
30082769,0.037652,-0.023477,-0.006596,0.013372,-0.007962


In [10]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,1,20,1,-0.044800,-0.019390,-0.031851,0.030554,-0.040778,-0.001448,-0.009343,0.004232,-0.025161,-0.028182
1,1,20,1,0.043076,-0.025291,-0.006318,0.018357,-0.021721,0.033837,-0.038700,0.039421,-0.034426,-0.041416
2,1,20,1,-0.043862,-0.046132,0.047778,-0.008464,0.017775,0.039120,-0.046248,-0.046503,0.019543,-0.020489
3,1,39,1,0.037853,0.040882,-0.035817,-0.036202,0.043400,0.005398,-0.004463,-0.022102,0.008920,0.026426
4,1,40,1,-0.044382,0.048629,-0.025343,-0.041154,-0.023619,0.018380,-0.037460,0.043368,0.009137,0.038793


In [11]:
df['product_category'].max()

18

In [12]:
# product_category embedding后 作为特征
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['product_category_0', 'product_category_1', 
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

(30082800, 1)
300828/300828 [==============================] - 3s 9us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4
0,1,20,1,-0.044800,-0.019390,-0.031851,0.030554,-0.040778,-0.001448,-0.009343,0.004232,-0.025161,-0.028182,-0.015297,0.001715,0.032569,0.027103,-0.003534
1,1,20,1,0.043076,-0.025291,-0.006318,0.018357,-0.021721,0.033837,-0.038700,0.039421,-0.034426,-0.041416,0.017035,0.035791,-0.014875,-0.024798,-0.026263
2,1,20,1,-0.043862,-0.046132,0.047778,-0.008464,0.017775,0.039120,-0.046248,-0.046503,0.019543,-0.020489,-0.015297,0.001715,0.032569,0.027103,-0.003534
3,1,39,1,0.037853,0.040882,-0.035817,-0.036202,0.043400,0.005398,-0.004463,-0.022102,0.008920,0.026426,-0.015297,0.001715,0.032569,0.027103,-0.003534
4,1,40,1,-0.044382,0.048629,-0.025343,-0.041154,-0.023619,0.018380,-0.037460,0.043368,0.009137,0.038793,0.017035,0.035791,-0.014875,-0.024798,-0.026263


In [13]:
df['advertiser_id'].max()

62965

In [14]:
# advertiser_id embedding后 作为特征
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(62966, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['advertiser_id_0', 'advertiser_id_1', 
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

300828/300828 [==============================] - 2s 7us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,20,1,-0.044800,-0.019390,-0.031851,0.030554,-0.040778,-0.001448,-0.009343,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,0.030851,-0.023410,0.020876,0.027672,0.029498
1,1,20,1,0.043076,-0.025291,-0.006318,0.018357,-0.021721,0.033837,-0.038700,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.033420,-0.042072,0.049573,0.038932,-0.041535
2,1,20,1,-0.043862,-0.046132,0.047778,-0.008464,0.017775,0.039120,-0.046248,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.019325,-0.047091,0.022594,0.047237,-0.004500
3,1,39,1,0.037853,0.040882,-0.035817,-0.036202,0.043400,0.005398,-0.004463,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.025302,0.041088,-0.049014,0.032754,-0.013170
4,1,40,1,-0.044382,0.048629,-0.025343,-0.041154,-0.023619,0.018380,-0.037460,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.015037,-0.022605,0.024558,-0.040403,-0.015491


In [15]:
# 对齐 为lstm作准备 对齐成(-1, 30, 22)
def check_matrix(x):
    if x.shape[0] >= 30:
        return x.iloc[:30, :]
    else:
        tmp1 = np.zeros((30 - x.shape[0], x.shape[1]))
        tmp1 = pd.DataFrame(tmp1)
        tmp1.columns = x.columns
        return pd.concat([x, tmp1], axis=0)
X = X.groupby('user_id').apply(check_matrix)
X = X.reset_index(drop=True)
X = X.drop('user_id', axis = 1)
X

,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,20.0,1.0,-0.044800,-0.019390,-0.031851,0.030554,-0.040778,-0.001448,-0.009343,0.004232,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,0.030851,-0.023410,0.020876,0.027672,0.029498
1,20.0,1.0,0.043076,-0.025291,-0.006318,0.018357,-0.021721,0.033837,-0.038700,0.039421,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.033420,-0.042072,0.049573,0.038932,-0.041535
2,20.0,1.0,-0.043862,-0.046132,0.047778,-0.008464,0.017775,0.039120,-0.046248,-0.046503,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.019325,-0.047091,0.022594,0.047237,-0.004500
3,39.0,1.0,0.037853,0.040882,-0.035817,-0.036202,0.043400,0.005398,-0.004463,-0.022102,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.025302,0.041088,-0.049014,0.032754,-0.013170
4,40.0,1.0,-0.044382,0.048629,-0.025343,-0.041154,-0.023619,0.018380,-0.037460,0.043368,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.015037,-0.022605,0.024558,-0.040403,-0.015491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26999995,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999997,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
# X.to_csv('data/train_preliminary/lstm_5.csv')

In [17]:
# 归一
X = preprocessing.MinMaxScaler().fit_transform(X)


In [18]:
X = np.array(X).reshape((900000, 30, 22))

In [19]:
# 预测性别
from keras.layers.core import Masking
X_train, X_test = X[:800000, :, :], X[800000:, :, :]
Y2_train, Y2_test = df1.iloc[:800000, 2:3], df1.iloc[800000:, 2:3]
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(30, 22)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y2_train, nb_epoch=50)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/50
800000/800000 [==============================] - 545s 681us/step - loss: 0.2101
Epoch 2/50
800000/800000 [==============================] - 495s 619us/step - loss: 0.1878
Epoch 3/50
800000/800000 [==============================] - 497s 621us/step - loss: 0.1756
Epoch 4/50
800000/800000 [==============================] - 497s 621us/step - loss: 0.1682
Epoch 5/50
800000/800000 [==============================] - 496s 619us/step - loss: 0.1633
Epoch 6/50
800000/800000 [==============================] - 496s 620us/step - loss: 0.1593
Epoch 7/50
800000/800000 [==============================] - 496s 620us/step - loss: 0.1560
Epoch 8/50
800000/800000 [==============================] - 496s 621us/step - loss: 0.1532
Epoch 9/50
800000/800000 [==============================] - 496s 621us/step - loss: 0.1506
Epoch 10/50
800000/800000 [==============================] - 496s 620us/step - loss: 0.1485
Epoch 11/50
800000/800000 [==============================] - 496s 620us/step - loss: 0.14

In [20]:
Y2_train.shape

(800000, 1)

In [21]:
Y = model.predict(X_train)
Y = np.round(Y)
ans = 0
for i in range(800000):
    if Y[i] == Y2_train.iloc[i, 0]:
        ans += 1
print('accurency:', ans/800000)

accurency: 0.82508375


In [22]:
Y = model.predict(X_test)
Y = np.round(Y)
ans = 0
for i in range(100000):
    if Y[i] == Y2_test.iloc[i, 0]:
        ans += 1
print('accurency:', ans/100000)

accurency: 0.82118


In [38]:
Y1_train, Y1_test = df1.iloc[:800000, 1:2], df1.iloc[800000:, 1:2]
Y1_train = pd.get_dummies(Y1_train['age'])
Y1_train

,1,2,3,4,5,6,7,8,9,10
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
799995,0,0,0,0,1,0,0,0,0,0
799996,0,1,0,0,0,0,0,0,0,0
799997,0,0,0,0,0,0,1,0,0,0
799998,0,0,0,1,0,0,0,0,0,0


In [39]:
# 预测年龄
model2 = Sequential()
model2.add(Masking(mask_value=0., input_shape=(30, 22)))
model2.add(LSTM(32))
model2.add(Dense(10))
model2.compile(loss='MSE', optimizer='adam')
model2.fit(X_train, Y1_train, nb_epoch=20)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
800000/800000 [==============================] - 490s 613us/step - loss: 0.0849
Epoch 2/20
800000/800000 [==============================] - 494s 618us/step - loss: 0.0839
Epoch 3/20
800000/800000 [==============================] - 495s 619us/step - loss: 0.0834
Epoch 4/20
800000/800000 [==============================] - 495s 618us/step - loss: 0.0831
Epoch 5/20
800000/800000 [==============================] - 498s 622us/step - loss: 0.0828
Epoch 6/20
800000/800000 [==============================] - 495s 619us/step - loss: 0.0826
Epoch 7/20
800000/800000 [==============================] - 500s 625us/step - loss: 0.0824
Epoch 8/20
800000/800000 [==============================] - 502s 628us/step - loss: 0.0822
Epoch 9/20
800000/800000 [==============================] - 506s 633us/step - loss: 0.0821
Epoch 10/20
800000/800000 [==============================] - 528s 660us/step - loss: 0.0819
Epoch 11/20
800000/800000 [==============================] - 526s 657us/step - loss: 0.08

In [41]:
Y = model2.predict(X_train)
ans = 0
for i in range(800000):
    if np.argmax(Y[i]) == df1.iloc[i, 1] - 1:
        ans += 1
print('accurency:', ans/800000)

accurency: 0.2701


In [42]:
Y = model2.predict(X_test)
ans = 0
for i in range(100000):
    if np.argmax(Y[i]) == df1.iloc[i + 800000, 1] - 1:
        ans += 1
print('accurency:', ans/100000)

accurency: 0.26837
